In [1]:
import pandas as pd
import numpy as np

from pymongo import MongoClient

import re

#fixed issue with ascii encoding errors
import sys

reload(sys)
sys.setdefaultencoding("utf-8")
#TODO: countvectorizer (ignores punctuation), nltk tokenizer

from nltk.tokenize import *

In [2]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.POETRY
poets = db.POET
poems = db.POEM
rawpoet = pd.DataFrame(list(poets.find()))
rawpoem = pd.DataFrame(list(poems.find()))

### clean rawpoet table. At the moment, _id, poet_positions, & posyears (position years) column values appear unusable
### drop these columns for now

In [3]:
ctodrop = ['_id', 'poet_positions', 'poet_posyears']
rawpoet.drop(ctodrop, 1,inplace = True)

In [4]:
ctostr = ['movement_name','poet_bio','poet_name', 'poet_basicbio', 'poet_tags']

for i in ctostr:
    rawpoet[i] = rawpoet[i].map(lambda x: u''.join(x))

In [5]:
rawpoet.head()[:1]

,movement_name,movement_url,poet_basicbio,poet_bio,poet_dob2,poet_name,poet_poems_url,poet_tags,poet_url
0,Slam/Spoken Word,http://www.poets.org/poetsorg/poets?field_scho...,,Taylor Mali is a four-time National Poetry Sla...,NaN,Taylor Mali,http://www.poets.org/poetsorg/poems/406436,\n \n ...,http://www.poets.org/poetsorg/poet/taylor-mali


###the poet's movement name tag is very important, we need to clean (remove dup tags), separate out into columns, then merge back into a clean poet table

In [6]:
rawpoet['tags'] = rawpoet[['movement_name', 'poet_tags']].apply(lambda x: list(set(str(x))),axis=1)
rawpoet['tags'] = rawpoet[['movement_name', 'poet_tags']].apply(lambda x: list(set(x)),axis=1)
rawpoet['tags'] = rawpoet['tags'].apply(lambda x: x.replace('-','').replace(' ','').split("\n"))
rawpoet['tags']=rawpoet['tags'].map(lambda x: ''.join(x).split(","))

In [7]:
rawpoet.tags[:10]

0    [Slam/SpokenWord]
1       [Contemporary]
2       [Objectivists]
3         [Surrealism]
4         [Surrealism]
5         [Surrealism]
6         [Surrealism]
7       [Objectivists]
8       [Objectivists]
9       [Objectivists]
Name: tags, dtype: object

In [8]:
ptags = rawpoet['tags'].apply(pd.Series)

In [9]:
ptags.columns = ['primary_tag', 'tag2', 'tag3','tag4']
ptags.head()

,primary_tag,tag2,tag3,tag4
0,Slam/SpokenWord,NaN,NaN,NaN
1,Contemporary,NaN,NaN,NaN
2,Objectivists,NaN,NaN,NaN
3,Surrealism,NaN,NaN,NaN
4,Surrealism,NaN,NaN,NaN


##now we need to look at poet births/deaths

In [10]:
plife = rawpoet['poet_dob2'].apply(pd.Series)

In [11]:
plife.columns = ['yr_born', 'yr_died']
plife.head()

,yr_born,yr_died
0,NaN,NaN
1,NaN,NaN
2,1942,1997
3,1926,2001
4,1926,NaN


poet_basicbio contains a secondary source of birth and death years. It also contains where the poet was born; city, state and country. We need to extract this information.

In [12]:
rawpoet['basic_bio']=rawpoet['poet_basicbio'].apply(lambda x: x.replace('-',''))

#extract anything that is a number
rawpoet['birthdeath'] = rawpoet['basic_bio'].map(lambda x: re.findall(r'\d+', x))
#extract non-numeric characters
rawpoet['birthplace'] = rawpoet['basic_bio'].map(lambda x: re.findall(r'[^\d]+', x))
rawpoet['birthplace'] = rawpoet['birthplace'].apply(lambda x: u''.join(x))

In [13]:
rawpoet['citystate']=rawpoet['birthplace'].map(lambda x: x.split(","))
pbirth = rawpoet['citystate'].apply(pd.Series)

In [16]:
pbirth.columns = ['citystate','country','country2']
pbirth['citystate'] = pbirth['citystate'].map(lambda x: x.split("         "))
pstate = pbirth['citystate'].apply(pd.Series)
pstate.fillna('', inplace =True)

In [17]:
pstate.columns = ['first','second','third','fourth','fifth']
pstate.drop('fifth',axis = 1, inplace = True)

##TODO: write function to do the below

In [18]:
pstate['state'] = pstate['second'].map(lambda x: str(x).replace(' ','') if len(str(x).replace(' ','')) == 2 else '')
pstate['city'] = pstate['second'].map(lambda x: str(x) if len(str(x).replace(' ','')) > 2 else '')
pstate['state2']=pstate['third'].map(lambda x: str(x).replace(' ','') if len(str(x).replace(' ','')) == 2 else '')
pstate['city2'] = pstate['third'].map(lambda x: str(x) if len(str(x).replace(' ','')) > 2 else '')
pstate['state3']=pstate['fourth'].map(lambda x: str(x).replace(' ','') if len(str(x).replace(' ','')) == 2 else '')
pstate['city3']=pstate['fourth'].map(lambda x: str(x) if len(str(x).replace(' ','')) > 2 else '')
pstate['state4']=pstate['first'].map(lambda x: str(x).replace(' ','') if len(str(x).replace(' ','')) == 2 else '')
pstate['city4']=pstate['first'].map(lambda x: str(x) if len(str(x).replace(' ','')) > 2 else '')

In [19]:
pstate.drop(['first','second','third','fourth'],axis = 1, inplace = True)

In [20]:
pstate['city'] = pstate['city']+pstate['city2']+pstate['city3']+pstate['city4']
pstate['state'] = pstate['state']+pstate['state2']+pstate['state3']+pstate['state4']
pstate['city']=pstate['city'].map(lambda x: x.lstrip().rstrip())
pstate['state']=pstate['state'].map(lambda x: x.lstrip().rstrip())

In [21]:
pstate.drop(['city2','city3','city4','state2','state3','state4'], axis=1, inplace = True)
pbirth.drop('citystate', axis = 1, inplace = True)

In [22]:
pbirth.fillna('',inplace=True)
pbio = pd.concat([pstate,pbirth],axis=1)

In [23]:
pbio['country2']=pbio['country2'].map(lambda x: x.lstrip().rstrip())
pbio['country']=pbio['country'].map(lambda x: x.lstrip().rstrip())

In [24]:
pbio.loc[pbio['city']=='Kensington', 'city'] = 'Kensington,London'
pbio.loc[pbio['city']=='Castledawson', 'city'] = 'Castledawson,County Derry'
pbio.loc[pbio['city']=='Kensington,London', 'country'] = 'United Kingdom'
pbio.loc[pbio['city']=='Castledawson,County Derry','country'] = 'Ireland' 

In [25]:
pbio.drop('country2', axis=1, inplace=True)

In [26]:
rawpoet.columns
coltokeep= ['poet_name','poet_url','poet_poems_url','tags','poet_bio']
poetdf = pd.concat([rawpoet[coltokeep],pbio,plife,ptags],axis=1)

In [27]:
poetdf['yr_born'].fillna(0, inplace = True)
poetdf['yr_died'].fillna(0, inplace = True)
# poetdf.head()
poetdf[['yr_born', 'yr_died']]= poetdf[['yr_born', 'yr_died']].astype(int)

In [28]:
poetdf['dead'] = np.where(poetdf['yr_died'] > 0,1,0)
poetdf['current_age'] = np.where(poetdf['dead'] == 0, (2015- poetdf['yr_born']), 0)
poetdf['age_dead'] = np.where(poetdf['dead'] == 1, (poetdf['yr_died']-poetdf['yr_born']), 0)
poetdf['current_age'] = np.where(poetdf['current_age'] == 2015, 0,poetdf['current_age'])

In [29]:
rawpoem.head()
poemdf = rawpoem[['poet_poems_url','poem_title', 'poem_yrpub', 'poem_text']]
resultdf = pd.merge(poetdf, poemdf, how='inner', on=['poet_poems_url'])

In [30]:
resultdf['tags'].value_counts()

[Contemporary]                                          704
[Modernism]                                             111
[LanguagePoetry]                                         68
[NewYorkSchool]                                          67
[Surrealism]                                             59
[Formalism]                                              59
[ConfessionalPoetry]                                     49
[JazzPoetry]                                             38
[Symbolists]                                             36
[HarlemRenaissance]                                      34
[Imagism, MetaphysicalPoet, Modernism, Objectivists]     34
[Beat]                                                   34
[Objectivists]                                           28
[MetaphysicalPoet, Modernism]                            21
[Imagism, MetaphysicalPoet, Modernism]                   20
[BlackMountain]                                          19
[ConcretePoetry]                        

In [31]:
resultdf.columns

Index([     u'poet_name',       u'poet_url', u'poet_poems_url',
                 u'tags',       u'poet_bio',          u'state',
                 u'city',        u'country',        u'yr_born',
              u'yr_died',    u'primary_tag',           u'tag2',
                 u'tag3',           u'tag4',           u'dead',
          u'current_age',       u'age_dead',     u'poem_title',
           u'poem_yrpub',      u'poem_text'],
      dtype='object')

Variables:

* year published
* total number of words
* avg words per new line
* total number of stanzas
* avg newlines per stanza
* if first word of each line starts with capital letter
* number of words with capital letters
* number of commas
* commas per line
* number of periods
* number of exclamations
* number of pronoun references
* if i, my is used bool
* type token ratio
* concrete words/words overall
* abstract/words overall
* number of named entities/total words
* alliteration
* number of sensory words
* topic modeling - political

In [106]:
text = resultdf.poem_text[12]
text

[u'\nIt was going on five in the morning\nThe ship of steam stretched its chain to shatter the windows\nAnd outside\nA glowworm\nLifted Paris like a leaf\nIt was only a long trembling scream\nA scream from the Maternity Hospital nearby\nFINIS FOUNDRY FANATIC\nBut whatever joy escaped in the exhalation of that pain\nIt seems to me that I was falling for a long time\nI still had my fist clenched around a handful of grass\nAnd suddenly that rustle of flowers and needles of ice\nThose green eyebrows that shooting-star pendulum\nFrom what depths was the bell actually able to rise again\nThe hermetic bell\nWhich nothing last night made me foresee would stop on this landing\nThe bell whose sides read\nUndine\nMoving to raise your spearheaded Sagittarius pedal\nYou had carved the infallible signs\nOf my enchantment\nWith a dagger whose coral handle forks into infinity\nSo that your blood and mine \nWould become one',
 u'\n']

In [113]:
text[0]

u'\nIt was going on five in the morning\nThe ship of steam stretched its chain to shatter the windows\nAnd outside\nA glowworm\nLifted Paris like a leaf\nIt was only a long trembling scream\nA scream from the Maternity Hospital nearby\nFINIS FOUNDRY FANATIC\nBut whatever joy escaped in the exhalation of that pain\nIt seems to me that I was falling for a long time\nI still had my fist clenched around a handful of grass\nAnd suddenly that rustle of flowers and needles of ice\nThose green eyebrows that shooting-star pendulum\nFrom what depths was the bell actually able to rise again\nThe hermetic bell\nWhich nothing last night made me foresee would stop on this landing\nThe bell whose sides read\nUndine\nMoving to raise your spearheaded Sagittarius pedal\nYou had carved the infallible signs\nOf my enchantment\nWith a dagger whose coral handle forks into infinity\nSo that your blood and mine \nWould become one'

In [94]:
ltext = []
# s.decode('unicode_escape')
for i in text:
    i.decode('unicode_escape')
    i = re.sub(u"\u2018", "'", i)
    i = re.sub(u"\u2019", "'", i)
    i = re.sub(u"\u2014", "-", i)
#     i = re.sub(u"\'", "", i)
    ltext.append(i)    

In [95]:
ltext

[u'\nAmidst the too much that we buy and throw\naway and the far too much we wrap it in,\nthe bear found a few items of special\ninterest--a honeydew rind, a used tampon,\nthe bone from a leg of lamb. He\'d rock back\nlightly onto his rear paws and slash\nopen a plastic bag, and then his nose--\njammed almost with a surfeit of rank\nand likely information, for he would pause--\nand then his whole dowsing snout would\ninsinuate itself a little way\ninside. By now he\'d have hunched his weight \nforward slightly, and then he\'d snatch it back,\ntrailed by some tidbit in his teeth. He\'d look\naround. What a good boy am he.\nThe guardian of the dump was used\nto this and not amused. "He\'ll drag that shit\nevery which damn way," he grumbled\nwho\'d dozed and scraped a pit to keep that shit\nwhere the town paid to contain it.\nThe others of us looked and looked. "City\nfolks like you don\'t get to see this often,"\none year-round resident accused me.\nSome winter I\'ll bring him down to le